In [2]:
import sys

In [1]:
import torch
torch.manual_seed(1)
import numpy as np
import csv
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
with open('calories.csv', mode='r', encoding='utf-8') as file:
    data = csv.reader(file)
    next(data)
    X = []
    #userid, male, female, age, height,weight, duration, heartrate, bodytemp, colories
    for line in data:
        gender = line[1]
        values = line[2:9]
        if gender == 'male':
            hot_encoding = [0, 1]
        else:
            hot_encoding = [1, 0]
        lineValues = hot_encoding + [float(a) for a in values]
        X.append(lineValues)
        
dataset = torch.tensor(X)

In [3]:
dataset

tensor([[  0.0000,   1.0000,  68.0000,  ..., 105.0000,  40.8000, 231.0000],
        [  1.0000,   0.0000,  20.0000,  ...,  94.0000,  40.3000,  66.0000],
        [  0.0000,   1.0000,  69.0000,  ...,  88.0000,  38.7000,  26.0000],
        ...,
        [  1.0000,   0.0000,  43.0000,  ...,  90.0000,  40.1000,  75.0000],
        [  0.0000,   1.0000,  78.0000,  ...,  84.0000,  38.3000,  11.0000],
        [  0.0000,   1.0000,  63.0000,  ...,  92.0000,  40.5000,  98.0000]])

In [33]:
len(dataset)

15000

In [34]:
dataset.shape

torch.Size([15000, 9])

In [4]:
X = dataset[:, :-1]
Y = dataset[:, -1].reshape(-1, 1)

In [5]:
X.shape


torch.Size([15000, 8])

In [13]:
Y.shape

torch.Size([15000, 1])

In [6]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

In [17]:
X_train.shape

torch.Size([9000, 8])

In [19]:
X_val.shape

torch.Size([3000, 8])

In [21]:
Y_test.shape

torch.Size([3000, 1])

In [7]:
W = torch.randn(9000,8)

In [8]:
print(W)

tensor([[-1.5256, -0.7502, -0.6540,  ..., -0.6092, -0.9798, -1.6091],
        [-0.7121,  0.3037, -0.7773,  ...,  1.6871,  0.2284,  0.4676],
        [-0.6970, -1.1608,  0.6995,  ...,  0.2444, -0.6629,  0.8073],
        ...,
        [ 0.5711,  1.4877, -1.2802,  ...,  0.3878, -1.4014, -0.2838],
        [-0.0591,  2.2088, -1.4102,  ...,  0.0139,  0.8020,  1.8881],
        [ 0.2883,  0.0562,  1.3478,  ..., -0.9728,  0.4575,  0.7477]])


In [9]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler2 = StandardScaler()
scaler2.fit(Y_train)

StandardScaler()

In [10]:
X_train_norm = scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)
X_train_norm = torch.from_numpy(X_train_norm)
X_val_norm = torch.from_numpy(X_val_norm)
X_test_norm = torch.from_numpy(X_test_norm)
Y_train_norm = scaler2.transform(Y_train)
Y_val_norm = scaler2.transform(Y_val)
Y_test_norm = scaler2.transform(Y_test)
Y_train_norm = torch.from_numpy(Y_train_norm)
Y_val_norm = torch.from_numpy(Y_val_norm)
Y_test_norm = torch.from_numpy(Y_test_norm)

In [11]:
X_train_norm

tensor([[ 0.9918, -0.9918,  0.1978,  ...,  1.0346,  0.2729,  0.8769],
        [-1.0083,  1.0083, -1.2781,  ..., -1.3699, -1.9218, -1.2902],
        [-1.0083,  1.0083,  0.6110,  ...,  0.4335,  1.0044,  0.3670],
        ...,
        [-1.0083,  1.0083,  0.8472,  ..., -0.8890, -0.5632, -0.6528],
        [-1.0083,  1.0083, -0.4516,  ..., -0.4081,  0.1683,  0.2395],
        [-1.0083,  1.0083, -0.9829,  ...,  0.0728, -0.4587,  0.6219]],
       dtype=torch.float64)

In [120]:
W = torch.randn(8, requires_grad=True , dtype=torch.float64)
B = torch.randn(1, requires_grad=True)

In [121]:
Y_train_norm_squeezed = torch.squeeze(Y_train_norm)
Y_val_norm_squeezed = torch.squeeze(Y_val_norm)

#Forward pass

In [126]:
epoch = 10000
patience = 5
no_imp = 0
best_val_loss = float('inf')
for i in range(epoch):
    y_pred = torch.matmul(X_train_norm,W.t()) + B
    loss=torch.mean((y_pred - Y_train_norm_squeezed)**2)
    W.grad = None
    loss.backward()
    W.data = W.data - (0.001 * W.grad)
    B.data = B.data - (0.001 * B.grad)
    with torch.no_grad():
        y_pred_val = torch.matmul(X_val_norm,W.t()) + B
        loss_val = torch.mean((y_pred_val - Y_val_norm_squeezed)**2)
    if loss_val < best_val_loss:
        best_val_loss = loss_val
        no_imp = 0
    else:
        no_imp += 1

    if no_imp >= patience:
        print("Early stopping!")
        break
    print(f'iteration: {i} Loss: {loss} Val Loss: {loss_val}')

iteration: 0 Loss: 0.42879981865501765 Val Loss: 0.42552442639045635
iteration: 1 Loss: 0.4263867114691963 Val Loss: 0.422350822216107
iteration: 2 Loss: 0.4231808614511047 Val Loss: 0.4184178522742405
iteration: 3 Loss: 0.4192070055206926 Val Loss: 0.4137563458347498
iteration: 4 Loss: 0.41449604942134516 Val Loss: 0.4084029734720252
iteration: 5 Loss: 0.40908475699857866 Val Loss: 0.40240004828167747
iteration: 6 Loss: 0.4030155523031009 Val Loss: 0.39579505514407415
iteration: 7 Loss: 0.3963360471531598 Val Loss: 0.38864041922808956
iteration: 8 Loss: 0.3890988110991068 Val Loss: 0.3809929387578657
iteration: 9 Loss: 0.3813608021994912 Val Loss: 0.37291343275707983
iteration: 10 Loss: 0.37318301565009504 Val Loss: 0.364466233928647
iteration: 11 Loss: 0.36462997610984993 Val Loss: 0.35571865793033736
iteration: 12 Loss: 0.35576920642156046 Val Loss: 0.34674045328006986
iteration: 13 Loss: 0.34667067696120824 Val Loss: 0.33760325609823677
iteration: 14 Loss: 0.3374062601114245 Val Lo

In [127]:
with torch.no_grad():
    y_test = torch.matmul(X_test_norm,W.t()) + B

In [128]:
y_test

tensor([-0.7973,  0.8172, -0.1697,  ..., -0.0863,  0.1420,  0.0813],
       dtype=torch.float64)

In [129]:
Y_test_norm

tensor([[-0.6501],
        [ 1.5814],
        [-0.3451],
        ...,
        [-0.3451],
        [ 0.2650],
        [ 0.2329]], dtype=torch.float64)

In [130]:
loss_test = torch.mean((y_test - Y_test_norm)**2)

In [62]:
y_test.reshape(1,-1)


tensor([[-0.5036,  1.3454, -0.2837,  ...,  0.2351,  0.1875,  0.3446]],
       dtype=torch.float64, grad_fn=<ViewBackward0>)

In [131]:
loss_test

tensor(2.0624, dtype=torch.float64)

In [132]:
mean2 = scaler2.mean_

In [133]:
var2 = scaler2.var_

In [134]:
std2 = var2**(1/2)

In [88]:
var2

array([3879.91752217])

In [89]:
std2

array([62.28898396])

In [135]:
y_end = y_test.numpy() * std2 + mean2

In [105]:
y_end += mean2

In [136]:
y_end.mean()

np.float64(83.58039156573848)

In [137]:
Y_test.mean()

tensor(90.6217)

In [138]:
y_end

array([ 38.83406963, 139.39527709,  77.92679892, ...,  83.11598378,
        97.3405812 ,  93.56074472], shape=(3000,))

In [139]:
Y_test

tensor([[ 48.],
        [187.],
        [ 67.],
        ...,
        [ 67.],
        [105.],
        [103.]])

In [142]:
#userid, male, female, age, height,weight, duration, heartrate, bodytemp, colories

In [184]:
X_predict = torch.Tensor([0,1,68,190,94,29,105,40]).double()

In [185]:
cal_pred = X_predict @ W.t() + B

In [186]:
cal_pred

tensor([88.5997], grad_fn=<AddBackward0>)

In [182]:
X[0]

tensor([  0.0000,   1.0000,  68.0000, 190.0000,  94.0000,  29.0000, 105.0000,
         40.8000])

In [183]:
Y[0]

tensor([231.])